In [16]:
# Load packages
import warnings

# Suppress warning messages
#warnings.filterwarnings('ignore')

import imdb
# import IMDbPY as imdb

import pandas as pd
import json

import nltk
import os
import re
import emoji
import pandas as pd

from nltk.corpus import stopwords
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from sklearn.decomposition import TruncatedSVD

from collections import Counter, defaultdict
from string import punctuation

from nltk.corpus import stopwords

sw = stopwords.words("english")

In [5]:
# Create IMDb Instance
ia = imdb.IMDb()

In [6]:
# Import movie names
# movienames = pd.read_csv(r'C:/UserData/z003cu8m/OneDrive - Siemens AG/Documents/Documents/Documents/Siemens/Computer Backup/Personal/MSADS/Text Mining/Week seven/movielist.csv')
movienames = pd.read_csv(r'C:/Users\elfek/datamining/Final/movielist.csv',quotechar='"')

movienames.head()

,Toy Story
0,Jumanji
1,Grumpier Old Men
2,Waiting to Exhale
3,Father of the Bride Part II
4,Heat


In [ ]:
movienameslist = movienames.values.tolist()

In [ ]:
# Pull information for movies
title = []
plot = []
genre = []

for name in movienameslist:
    movie = ia.search_movie(str(name))
    movie = movie[0]
    movie = ia.get_movie(movie.movieID)
    title.append(name)
    movieplot = movie.get('plot')
    plot.append(movieplot)
    moviegenre = movie.get('genre')
    genre.append('moviegenre')

In [ ]:
# Combine into dataframe
moviedata = list(zip(title, plot, genre))

In [3]:
# movie_meta = pd.read_csv(r'C:/UserData/z003cu8m/OneDrive - Siemens AG/Documents/Documents/Documents/Siemens/Computer Backup/Personal/MSADS/Text Mining/Week seven/movies_metadata.csv')
movie_meta = pd.read_csv(r'C:/Users/LadyBug/Documents/GitHub/Predicting-Movie-Rating/movies_data.csv',quotechar='"')

movie_meta.head()

<ipython-input-3-6e88f181a251>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_meta = pd.read_csv(r'C:/Users/LadyBug/Documents/GitHub/Predicting-Movie-Rating/movies_data.csv',quotechar='"')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
# extracting titles, genres and movie overview

movie_meta_new = movie_meta[["title", "genres", "overview"]]

#This is the first step to be able to convert the json. We need to replace single quotes with double quotes.
movie_meta_new = movie_meta_new.replace("'",'"', regex=True)

#Previously I had this as the first step but there is a warning/potential bug if we do it this way. I'm leaving it as a warning, lol.
#movie_meta_new['genres'] = movie_meta_new['genres'].str.replace("'",'"')


nice_genres =[]

for row in movie_meta_new['genres']:
    #Now that the genres has quotes it can be converted from a string to a json dictionary
    dict_row = json.loads(row)
    #After being converted to a json dictionary we can use a json pandas parser to make it into a dataframe
    df_row = pd.json_normalize(dict_row)
    #Some of the json fields are empty inside which causes an error once we finish all the parsings and try to add it to the list 
    if not df_row.empty:
        list_row = df_row["name"].to_list()
        nice_genres.append( list_row)
    #We have to add a null to the list where the json field is empty so we don't mess with the structure of the dataframe
    else:
        nice_genres.append(None)


movie_meta_new["nice_genres"]=nice_genres

movie_meta_new["nice_genres"].tail()


45461              [Drama, Family]
45462                      [Drama]
45463    [Action, Drama, Thriller]
45464                         None
45465                         None
Name: nice_genres, dtype: object

In [7]:
#Now we can drop those empty fields which I converted into nulls.
movie_meta_new["nice_genres"].dropna().tail()

45459            [Science Fiction]
45460     [Drama, Action, Romance]
45461              [Drama, Family]
45462                      [Drama]
45463    [Action, Drama, Thriller]
Name: nice_genres, dtype: object

In [9]:
# Removing secondary genres
movie_meta_new["singlegenre"] = movie_meta_new["nice_genres"].str[0]

In [14]:
# Combining categories and dropping some to limit the number of genres to 10
movie_meta_new['singlegenre'] = movie_meta_new['singlegenre'].replace({
    'Drama' : 'Drama/Mystery',
    'Action' : 'Action/War',
    'Mystery' : 'Drama/Mystery',
    'War' : 'Action/War',
    'Documentary' : 'Documentary/History',
    'History' : 'Documentary/History',
    'Horror' : 'Horror/Crime/Thriller',
    'Crime' : 'Horror/Crime/Thriller',
    'Thriller' : 'Horror/Crime/Thriller',
    'Animation' : 'Animation/Family',
    'Family' : 'Animation/Family',
    'Science Fiction' : 'Science Fiction/Fantasy',
    'Fantasy' : 'Science Fiction/Fantasy'   
})

movie_meta_new = movie_meta_new[~movie_meta_new['singlegenre'].isin(['Music', 'Western', 'TV Movie', 'Foreign', 'Carousel Productions', 'Aniplex'])]

In [18]:
# data cleaning

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison

def remove_stop(tokens) :
    # modify this function to remove stopwords
    return(tokens)
 
def remove_punctuation(text, punct_set= punctuation) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return(text)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


In [19]:
# apply the `pipeline` technique 

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

movie_meta_new["overview"] = movie_meta_new["overview"].apply(prepare,pipeline=my_pipeline)

<ipython-input-19-5113143150c4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_meta_new["overview"] = movie_meta_new["overview"].apply(prepare,pipeline=my_pipeline)
